## Data cleaning of pharmacy sales indications

In [2]:
import os
import glob

# Data handling and manipulation library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Set print options
pd.set_option('display.max_columns', None)

### some stuff for making pretty plots
from matplotlib import rcParams
from cycler import cycler
## Set plotting style and print options
sns.set_theme()
sns.set_style("whitegrid")
sns.set_context("paper") #Possible are paper, notebook, talk and poster
# Set figure format
wide_format, square_format = False, True
if wide_format:
    d = {'lines.linewidth': 2, 'axes.titlesize': 18, 'axes.labelsize': 18, 'xtick.labelsize': 12, 'ytick.labelsize': 12,\
     'legend.fontsize': 15, 'font.family': 'serif', 'figure.figsize': (9,6)}
if square_format:
    d = {'lines.linewidth': 2, 'axes.titlesize': 16, 'axes.labelsize': 14, 'xtick.labelsize': 10, 'ytick.labelsize': 10,\
     'legend.fontsize': 12, 'font.family': 'serif', 'figure.figsize': (6,6)}
    
d_colors = {'axes.prop_cycle': cycler(color = ['teal', 'navy', 'coral', 'plum', 'purple', 'olivedrab',\
         'black', 'red', 'cyan', 'yellow', 'khaki','lightblue'])}
rcParams.update(d)
rcParams.update(d_colors)

Get an overview of all the files in the indication insights folder

In [12]:
# Set the path to the data directory
data_dir = "../../../Indication Insights Data-20240905"

# List all files in the data directory
files = glob.glob(os.path.join(data_dir, '*.csv'))
file_names = os.listdir(data_dir)
file_names = [file.lower() for file in file_names if file.endswith('.csv')]

# Load all data files into a dictionary
df_dict = {}
for file in files:
    df = pd.read_csv(file, sep=';',encoding='iso-8859-1')
    df_dict[os.path.basename(file).lower().split('.csv')[0]] = df

keys = list(df_dict.keys())
file_names

['diagnosis_groups.csv',
 'diagnosis_groups_sks diagnoser.csv',
 'diagnosis_groups_spa.csv',
 'dosing.csv',
 'hospital_account.csv',
 'hospital_region.csv',
 'salgs_data_indication_insight_hospital.csv',
 'salgs_data_indication_insight_pharmacy.csv',
 'substancesofinterest.csv',
 'tabel_sks_diagnose_country.csv',
 'tabel_sks_diagnose_hospital.csv',
 'tabel_sks_diagnose_region.csv']

Clean 'salgs_data_indication_insight_pharmacy'

In [ ]:
choose_file = 'salgs_data_indication_insight_pharmacy'
sales_ins = df_dict[choose_file]